In [1]:
import argparse
import random
from sampler import data_sampler
from config import Config
import torch
from model.bert_encoder import Bert_Encoder
from model.dropout_layer import Dropout_Layer
from model.classifier import Softmax_Layer, Proto_Softmax_Layer
from data_loader import get_data_loader
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.cluster import KMeans
import collections
from copy import deepcopy
import os
# import wandb
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


def train_simple_model(config, encoder, dropout_layer, classifier, training_data, epochs, map_relid2tempid):
    data_loader = get_data_loader(config, training_data, shuffle=True)

    encoder.train()
    dropout_layer.train()
    classifier.train()

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam([
        {'params': encoder.parameters(), 'lr': 0.00001},
        {'params': dropout_layer.parameters(), 'lr': 0.00001},
        {'params': classifier.parameters(), 'lr': 0.001}
    ])
    for epoch_i in range(epochs):
        losses = []
        for step, batch_data in enumerate(data_loader):
            optimizer.zero_grad()
            labels, _, tokens = batch_data
            labels = labels.to(config.device)
            labels = [map_relid2tempid[x.item()] for x in labels]
            labels = torch.tensor(labels).to(config.device)

            tokens = torch.stack([x.to(config.device) for x in tokens],dim=0)
            reps = encoder(tokens)
            reps, _ = dropout_layer(reps)
            logits = classifier(reps)
            loss = criterion(logits, labels)

            losses.append(loss.item())
            loss.backward()
            optimizer.step()
        print(f"loss is {np.array(losses).mean()}")


def compute_jsd_loss(m_input):
    # m_input: the result of m times dropout after the classifier.
    # size: m*B*C
    m = m_input.shape[0]
    mean = torch.mean(m_input, dim=0)
    jsd = 0
    for i in range(m):
        loss = F.kl_div(F.log_softmax(mean, dim=-1), F.softmax(m_input[i], dim=-1), reduction='none')
        loss = loss.sum()
        jsd += loss / m
    return jsd


def contrastive_loss(hidden, labels):

    logsoftmax = nn.LogSoftmax(dim=-1)

    return -(logsoftmax(hidden) * labels).sum() / labels.sum()


def construct_hard_triplets(output, labels, relation_data):
    positive = []
    negative = []
    pdist = nn.PairwiseDistance(p=2)
    for rep, label in zip(output, labels):
        positive_relation_data = relation_data[label.item()]
        negative_relation_data = []
        for key in relation_data.keys():
            if key != label.item():
                negative_relation_data.extend(relation_data[key])
        positive_distance = torch.stack([pdist(rep.cpu(), p) for p in positive_relation_data])
        negative_distance = torch.stack([pdist(rep.cpu(), n) for n in negative_relation_data])
        positive_index = torch.argmax(positive_distance)
        negative_index = torch.argmin(negative_distance)
        positive.append(positive_relation_data[positive_index.item()])
        negative.append(negative_relation_data[negative_index.item()])


    return positive, negative


def train_first(config, encoder, dropout_layer, classifier, training_data, epochs, map_relid2tempid, new_relation_data):
    data_loader = get_data_loader(config, training_data, shuffle=True)

    encoder.train()
    dropout_layer.train()
    classifier.train()

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam([
        {'params': encoder.parameters(), 'lr': 0.00001},
        {'params': dropout_layer.parameters(), 'lr': 0.00001},
        {'params': classifier.parameters(), 'lr': 0.001}
    ])
    triplet_loss = nn.TripletMarginLoss(margin=1.0, p=2)
    for epoch_i in range(epochs):
        losses = []
        for step, (labels, _, tokens) in enumerate(data_loader):

            optimizer.zero_grad()

            logits_all = []
            tokens = torch.stack([x.to(config.device) for x in tokens], dim=0)
            labels = labels.to(config.device)
            origin_labels = labels[:]
            labels = [map_relid2tempid[x.item()] for x in labels]
            labels = torch.tensor(labels).to(config.device)
            reps = encoder(tokens)
            outputs,_ = dropout_layer(reps)
            positives,negatives = construct_hard_triplets(outputs, origin_labels, new_relation_data)

            for _ in range(config.f_pass):
                output, output_embedding = dropout_layer(reps)
                logits = classifier(output)
                logits_all.append(logits)

            positives = torch.cat(positives, 0).to(config.device)
            negatives = torch.cat(negatives, 0).to(config.device)
            anchors = outputs
            logits_all = torch.stack(logits_all)
            m_labels = labels.expand((config.f_pass, labels.shape[0]))  # m,B
            loss1 = criterion(logits_all.reshape(-1, logits_all.shape[-1]), m_labels.reshape(-1))
            loss2 = compute_jsd_loss(logits_all)
            tri_loss = triplet_loss(anchors, positives, negatives)
            loss = loss1 + loss2 + tri_loss

            loss.backward()
            losses.append(loss.item())
            optimizer.step()
        print(f"loss is {np.array(losses).mean()}")


def train_mem_model(config, encoder, dropout_layer, classifier, training_data, epochs, map_relid2tempid, new_relation_data,
                prev_encoder, prev_dropout_layer, prev_classifier, prev_relation_index):
    data_loader = get_data_loader(config, training_data, shuffle=True)

    encoder.train()
    dropout_layer.train()
    classifier.train()

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam([
        {'params': encoder.parameters(), 'lr': 0.00001},
        {'params': dropout_layer.parameters(), 'lr': 0.00001},
        {'params': classifier.parameters(), 'lr': 0.001}
    ])
    triplet_loss = nn.TripletMarginLoss(margin=1.0, p=2)
    distill_criterion = nn.CosineEmbeddingLoss()
    T = config.kl_temp
    for epoch_i in range(epochs):
        losses = []
        for step, (labels, _, tokens) in enumerate(data_loader):

            optimizer.zero_grad()

            logits_all = []
            tokens = torch.stack([x.to(config.device) for x in tokens], dim=0)
            labels = labels.to(config.device)
            origin_labels = labels[:]
            labels = [map_relid2tempid[x.item()] for x in labels]
            labels = torch.tensor(labels).to(config.device)
            reps = encoder(tokens)
            normalized_reps_emb = F.normalize(reps.view(-1, reps.size()[1]), p=2, dim=1)
            outputs,_ = dropout_layer(reps)
            if prev_dropout_layer is not None:
                prev_outputs, _ = prev_dropout_layer(reps)
                positives,negatives = construct_hard_triplets(prev_outputs, origin_labels, new_relation_data)
            else:
                positives, negatives = construct_hard_triplets(outputs, origin_labels, new_relation_data)

            for _ in range(config.f_pass):
                output, output_embedding = dropout_layer(reps)
                logits = classifier(output)
                logits_all.append(logits)

            positives = torch.cat(positives, 0).to(config.device)
            negatives = torch.cat(negatives, 0).to(config.device)
            anchors = outputs
            logits_all = torch.stack(logits_all)
            m_labels = labels.expand((config.f_pass, labels.shape[0]))  # m,B
            loss1 = criterion(logits_all.reshape(-1, logits_all.shape[-1]), m_labels.reshape(-1))
            loss2 = compute_jsd_loss(logits_all)
            tri_loss = triplet_loss(anchors, positives, negatives)
            loss = loss1 + loss2 + tri_loss

            if prev_encoder is not None:
                prev_reps = prev_encoder(tokens).detach()
                normalized_prev_reps_emb = F.normalize(prev_reps.view(-1, prev_reps.size()[1]), p=2, dim=1)

                feature_distill_loss = distill_criterion(normalized_reps_emb, normalized_prev_reps_emb,
                                                         torch.ones(tokens.size(0)).to(
                                                             config.device))
                loss += feature_distill_loss

            if prev_dropout_layer is not None and prev_classifier is not None:
                prediction_distill_loss = None
                dropout_output_all = []
                prev_dropout_output_all = []
                for i in range(config.f_pass):
                    output, _ = dropout_layer(reps)
                    prev_output, _ = prev_dropout_layer(reps)
                    dropout_output_all.append(output)
                    prev_dropout_output_all.append(output)
                    pre_logits = prev_classifier(output).detach()

                    pre_logits = F.softmax(pre_logits.index_select(1, prev_relation_index) / T, dim=1)

                    log_logits = F.log_softmax(logits_all[i].index_select(1, prev_relation_index) / T, dim=1)
                    if i == 0:
                        prediction_distill_loss = -torch.mean(torch.sum(pre_logits * log_logits, dim=1))
                    else:
                        prediction_distill_loss += -torch.mean(torch.sum(pre_logits * log_logits, dim=1))

                prediction_distill_loss /= config.f_pass
                loss += prediction_distill_loss
                dropout_output_all = torch.stack(dropout_output_all)
                prev_dropout_output_all = torch.stack(prev_dropout_output_all)
                mean_dropout_output_all = torch.mean(dropout_output_all, dim=0)
                mean_prev_dropout_output_all = torch.mean(prev_dropout_output_all,dim=0)
                normalized_output = F.normalize(mean_dropout_output_all.view(-1, mean_dropout_output_all.size()[1]), p=2, dim=1)
                normalized_prev_output = F.normalize(mean_prev_dropout_output_all.view(-1, mean_prev_dropout_output_all.size()[1]), p=2, dim=1)
                hidden_distill_loss = distill_criterion(normalized_output, normalized_prev_output,
                                                         torch.ones(tokens.size(0)).to(
                                                             config.device))
                loss += hidden_distill_loss

            loss.backward()
            losses.append(loss.item())
            optimizer.step()
        print(f"loss is {np.array(losses).mean()}")




def batch2device(batch_tuple, device):
    ans = []
    for var in batch_tuple:
        if isinstance(var, torch.Tensor):
            ans.append(var.to(device))
        elif isinstance(var, list):
            ans.append(batch2device(var))
        elif isinstance(var, tuple):
            ans.append(tuple(batch2device(var)))
        else:
            ans.append(var)
    return ans


def evaluate_strict_model(config, encoder, dropout_layer, classifier, test_data, seen_relations, map_relid2tempid):
    data_loader = get_data_loader(config, test_data, batch_size=1)
    encoder.eval()
    dropout_layer.eval()
    classifier.eval()
    n = len(test_data)

    correct = 0
    for step, batch_data in enumerate(data_loader):
        labels, _, tokens = batch_data
        labels = labels.to(config.device)
        labels = [map_relid2tempid[x.item()] for x in labels]
        labels = torch.tensor(labels).to(config.device)

        tokens = torch.stack([x.to(config.device) for x in tokens],dim=0)
        reps = encoder(tokens)
        reps, _ = dropout_layer(reps)
        logits = classifier(reps)

        seen_relation_ids = [rel2id[relation] for relation in seen_relations]
        seen_relation_ids = [map_relid2tempid[relation] for relation in seen_relation_ids]
        seen_sim = logits[:,seen_relation_ids].cpu().data.numpy()
        max_smi = np.max(seen_sim,axis=1)

        label_smi = logits[:,labels].cpu().data.numpy()

        if label_smi >= max_smi:
            correct += 1

    return correct/n


def select_data(config, encoder, dropout_layer, relation_dataset):
    data_loader = get_data_loader(config, relation_dataset, shuffle=False, drop_last=False, batch_size=1)
    features = []
    encoder.eval()
    dropout_layer.eval()
    for step, batch_data in enumerate(data_loader):
        labels, _, tokens = batch_data
        tokens = torch.stack([x.to(config.device) for x in tokens],dim=0)
        with torch.no_grad():
            feature = dropout_layer(encoder(tokens))[1].cpu()
        features.append(feature)

    features = np.concatenate(features)
    num_clusters = min(config.num_protos, len(relation_dataset))
    distances = KMeans(n_clusters=num_clusters, random_state=0).fit_transform(features)

    memory = []
    for k in range(num_clusters):
        sel_index = np.argmin(distances[:, k])
        instance = relation_dataset[sel_index]
        memory.append(instance)
    return memory


def get_proto(config, encoder, dropout_layer, relation_dataset):
    data_loader = get_data_loader(config, relation_dataset, shuffle=False, drop_last=False, batch_size=1)
    features = []
    encoder.eval()
    dropout_layer.eval()
    for step, batch_data in enumerate(data_loader):
        labels, _, tokens = batch_data
        tokens = torch.stack([x.to(config.device) for x in tokens],dim=0)
        with torch.no_grad():
            feature = dropout_layer(encoder(tokens))[1]
        features.append(feature)
    features = torch.cat(features, dim=0)
    proto = torch.mean(features, dim=0, keepdim=True).cpu()
    standard = torch.sqrt(torch.var(features, dim=0)).cpu()
    return proto, standard


def generate_relation_data(protos, relation_standard):
    relation_data = {}
    relation_sample_nums = 10
    for id in protos.keys():
        relation_data[id] = []
        difference = np.random.normal(loc=0, scale=1, size=relation_sample_nums)
        for diff in difference:
            relation_data[id].append(protos[id] + diff * relation_standard[id])
    return relation_data


def generate_current_relation_data(config, encoder, dropout_layer, relation_dataset):
    data_loader = get_data_loader(config, relation_dataset, shuffle=False, drop_last=False, batch_size=1)
    relation_data = []
    encoder.eval()
    dropout_layer.eval()
    for step, batch_data in enumerate(data_loader):
        labels, _, tokens = batch_data
        tokens = torch.stack([x.to(config.device) for x in tokens],dim=0)
        with torch.no_grad():
            feature = dropout_layer(encoder(tokens))[1].cpu()
        relation_data.append(feature)
    return relation_data

from transformers import  BertTokenizer
def data_augmentation(config, encoder, train_data, prev_train_data):
    expanded_train_data = train_data[:]
    expanded_prev_train_data = prev_train_data[:]
    encoder.eval()
    all_data = train_data + prev_train_data
    tokenizer = BertTokenizer.from_pretrained(config.bert_path, additional_special_tokens=["[E11]", "[E12]", "[E21]", "[E22]"])
    entity_index = []
    entity_mention = []
    for sample in all_data:
        e11 = sample['tokens'].index(30522)
        e12 = sample['tokens'].index(30523)
        e21 = sample['tokens'].index(30524)
        e22 = sample['tokens'].index(30525)
        entity_index.append([e11,e12])
        entity_mention.append(sample['tokens'][e11+1:e12])
        entity_index.append([e21,e22])
        entity_mention.append(sample['tokens'][e21+1:e22])

    data_loader = get_data_loader(config, all_data, shuffle=False, drop_last=False, batch_size=1)
    features = []
    encoder.eval()
    for step, batch_data in enumerate(data_loader):
        labels, _, tokens = batch_data
        tokens = torch.stack([x.to(config.device) for x in tokens],dim=0)
        with torch.no_grad():
            feature = encoder(tokens)
        feature1, feature2 = torch.split(feature, [config.encoder_output_size,config.encoder_output_size], dim=1)
        features.append(feature1)
        features.append(feature2)
    features = torch.cat(features, dim=0)
    # similarity_matrix = F.cosine_similarity(features.unsqueeze(1), features.unsqueeze(0), dim=-1)
    similarity_matrix = []
    for i in range(len(features)):
        similarity_matrix.append([0]*len(features))

    for i in range(len(features)):
        for j in range(i,len(features)):
            similarity = F.cosine_similarity(features[i],features[j],dim=0)
            similarity_matrix[i][j] = similarity
            similarity_matrix[j][i] = similarity

    similarity_matrix = torch.tensor(similarity_matrix).to(config.device)
    zero = torch.zeros_like(similarity_matrix).to(config.device)
    diag = torch.diag_embed(torch.diag(similarity_matrix))
    similarity_matrix -= diag
    similarity_matrix = torch.where(similarity_matrix<0.95, zero, similarity_matrix)
    nonzero_index = torch.nonzero(similarity_matrix)
    expanded_train_count = 0

    for origin, replace in nonzero_index:
        sample_index = int(origin/2)
        sample = all_data[sample_index]
        if entity_mention[origin] == entity_mention[replace]:
            continue
        new_tokens = sample['tokens'][:entity_index[origin][0]+1] + entity_mention[replace] + sample['tokens'][entity_index[origin][1]:]
        if len(new_tokens) < config.max_length:
            new_tokens = new_tokens + [0]*(config.max_length-len(new_tokens))
        else:
            new_tokens = new_tokens[:config.max_length]

        new_sample = {
            'relation': sample['relation'],
            'neg_labels': sample['neg_labels'],
            'tokens': new_tokens
        }
        if sample_index < len(train_data) and expanded_train_count < 5 * len(train_data):
            expanded_train_data.append(new_sample)
            expanded_train_count += 1
        else:
            expanded_prev_train_data.append(new_sample)
    return expanded_train_data, expanded_prev_train_data


/home/thanh/miniconda3/envs/sckd/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class args:
    task = "fewrel"
    shot = 5
    config = 'config.ini'
    


In [3]:
config = Config(args.config)
config.device = torch.device(config.device)
config.n_gpu = torch.cuda.device_count()
config.batch_size_per_step = int(config.batch_size / config.gradient_accumulation_steps)

config.task = args.task
config.shot = args.shot
config.step1_epochs = 5
config.step2_epochs = 15
config.step3_epochs = 20
config.temperature = 0.08

if config.task == "FewRel":
    config.relation_file = "data/fewrel/relation_name.txt"
    config.rel_index = "data/fewrel/rel_index.npy"
    config.rel_feature = "data/fewrel/rel_feature.npy"
    config.rel_des_file = "data/fewrel/relation_description.txt"
    config.num_of_relation = 80
    if config.shot == 5:
        config.rel_cluster_label = "data/fewrel/CFRLdata_10_100_10_5/rel_cluster_label_0.npy"
        config.training_file = "data/fewrel/CFRLdata_10_100_10_5/train_0.txt"
        config.valid_file = "data/fewrel/CFRLdata_10_100_10_5/valid_0.txt"
        config.test_file = "data/fewrel/CFRLdata_10_100_10_5/test_0.txt"
    elif config.shot == 10:
        config.rel_cluster_label = "data/fewrel/CFRLdata_10_100_10_10/rel_cluster_label_0.npy"
        config.training_file = "data/fewrel/CFRLdata_10_100_10_10/train_0.txt"
        config.valid_file = "data/fewrel/CFRLdata_10_100_10_10/valid_0.txt"
        config.test_file = "data/fewrel/CFRLdata_10_100_10_10/test_0.txt"
    else:
        config.rel_cluster_label = "data/fewrel/CFRLdata_10_100_10_2/rel_cluster_label_0.npy"
        config.training_file = "data/fewrel/CFRLdata_10_100_10_2/train_0.txt"
        config.valid_file = "data/fewrel/CFRLdata_10_100_10_2/valid_0.txt"
        config.test_file = "data/fewrel/CFRLdata_10_100_10_2/test_0.txt"
else:
    config.relation_file = "data/tacred/relation_name.txt"
    config.rel_index = "data/tacred/rel_index.npy"
    config.rel_feature = "data/tacred/rel_feature.npy"
    config.num_of_relation = 41
    if config.shot == 5:
        config.rel_cluster_label = "data/tacred/CFRLdata_10_100_10_5/rel_cluster_label_0.npy"
        config.training_file = "data/tacred/CFRLdata_10_100_10_5/train_0.txt"
        config.valid_file = "data/tacred/CFRLdata_10_100_10_5/valid_0.txt"
        config.test_file = "data/tacred/CFRLdata_10_100_10_5/test_0.txt"
    else:
        config.rel_cluster_label = "data/tacred/CFRLdata_10_100_10_10/rel_cluster_label_0.npy"
        config.training_file = "data/tacred/CFRLdata_10_100_10_10/train_0.txt"
        config.valid_file = "data/tacred/CFRLdata_10_100_10_10/valid_0.txt"
        config.test_file = "data/tacred/CFRLdata_10_100_10_10/test_0.txt"


In [4]:
sampler = data_sampler(config=config, seed=config.seed+100)


[7 6 5 2 1 0 3 4]


In [5]:
data  = []
for steps, (training_data, valid_data, test_data, current_relations, historic_test_data, seen_relations) in enumerate(sampler):
            print(current_relations)
            data.append((training_data, valid_data, test_data, current_relations, historic_test_data, seen_relations))    

['person countries of residence', 'organization top members employees', 'organization member of', 'person origin', 'person title', 'organization country of headquarters']
['person stateorprovinces of residence', 'person date of death', 'organization number of employees members', 'person alternate names', 'person spouse']
['person date of birth', 'person stateorprovince of birth', 'person parents', 'person employee of', 'person stateorprovince of death']
['person cities of residence', 'person schools attended', 'person country of death', 'person children', 'person charges']
['organization subsidiaries', 'organization parents', 'organization alternate names', 'organization city of headquarters', 'person siblings']
['person country of birth', 'organization website', 'organization shareholders', 'organization dissolved', 'organization founded by']
['person cause of death', 'organization political religious affiliation', 'organization stateorprovince of headquarters', 'person other family',

In [4]:
config.device = 'cpu'

In [5]:
from model.bert_encoder import Bert_EncoderMLM
model = Bert_EncoderMLM(config)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relations

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [7]:
a = ['this is some text' , 'this is some another text']

In [8]:
inputs = tokenizer(a, return_tensors='pt', padding=True)

In [9]:
outputs = model(inputs['input_ids'])

No mask token found in the input sequence
No mask token found in the input sequence


In [10]:
outputs[0].shape

torch.Size([2, 768])

In [6]:
train_data = []

In [7]:
for j in range(len(data)):
    train_data = []
    for i in range(len(list(data[j][0].values()))):
        train_data.extend(list(data[j][0].values())[i])
    cnt = 0
    for x in train_data:
        if 30524 not in x['tokens']:
            cnt +=1
            print(x['tokens'])
    if cnt > 0:
        print(j)
        print(cnt)

[101, 2268, 1011, 5757, 1011, 6021, 2102, 2692, 2549, 1024, 5354, 1024, 2184, 4012, 1028, 2626, 1024, 9587, 14945, 25353, 10936, 3669, 25886, 1026, 25353, 10936, 1030, 20643, 9006, 1028, 8299, 1024, 1013, 1013, 7479, 29337, 28251, 8586, 5358, 1013, 3422, 1029, 1058, 1027, 1053, 2063, 2620, 4328, 2629, 2497, 2860, 2683, 16409, 9587, 14945, 25353, 10936, 3669, 25886, 1026, 25353, 10936, 1030, 20643, 9006, 1028, 8299, 1024, 1013, 1013, 2739, 15396, 5643, 9006, 1013, 2739, 1013, 4021, 5643, 1003, 1016, 24700, 7974, 2015, 1013, 6027, 1013, 2466, 1013, 17350, 23809, 2100, 28332, 4274, 2003, 6179, 2005, 2437, 1996, 3606, 2124, 1010, 2758, 2852, 24404, 15222, 2099, 1036, 1036, 15490, 17761, 1010, 2238, 1015, 1011, 1048, 15185, 1011, 30522, 16595, 8067, 30523, 1011, 25269, 2497, 1011, 1011, 102]
5
1


SyntaxError: invalid syntax (942630328.py, line 4)

In [67]:
import numpy as np
np.argwhere(tokens == 30524).size

0

In [53]:
tokens = np.array(tokens)

In [49]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(config.bert_path, additional_special_tokens=["[E11]", "[E12]", "[E21]", "[E22]"])
tokens = [101, 2268, 1011, 5757, 1011, 6021, 2102, 2692, 2549, 1024, 5354, 1024, 2184, 4012, 1028, 2626, 1024, 9587, 14945, 25353, 10936, 3669, 25886, 1026, 25353, 10936, 1030, 20643, 9006, 1028, 8299, 1024, 1013, 1013, 7479, 29337, 28251, 8586, 5358, 1013, 3422, 1029, 1058, 1027, 1053, 2063, 2620, 4328, 2629, 2497, 2860, 2683, 16409, 9587, 14945, 25353, 10936, 3669, 25886, 1026, 25353, 10936, 1030, 20643, 9006, 1028, 8299, 1024, 1013, 1013, 2739, 15396, 5643, 9006, 1013, 2739, 1013, 4021, 5643, 1003, 1016, 24700, 7974, 2015, 1013, 6027, 1013, 2466, 1013, 17350, 23809, 2100, 28332, 4274, 2003, 6179, 2005, 2437, 1996, 3606, 2124, 1010, 2758, 2852, 24404, 15222, 2099, 1036, 1036, 15490, 17761, 1010, 2238, 1015, 1011, 1048, 15185, 1011, 30522, 16595, 8067, 30523, 1011, 25269, 2497, 1011, 1011, 102]
print(tokenizer.convert_ids_to_tokens(tokens))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


['[CLS]', '2009', '-', '06', '-', '03', '##t', '##0', '##4', ':', '59', ':', '10', 'com', '>', 'wrote', ':', 'mo', '##hd', 'sy', '##az', '##li', 'mahmud', '<', 'sy', '##az', '@', 'yahoo', '##com', '>', 'http', ':', '/', '/', 'www', '##you', '##tub', '##ec', '##om', '/', 'watch', '?', 'v', '=', 'q', '##e', '##8', '##mi', '##5', '##b', '##w', '##9', '##dc', 'mo', '##hd', 'sy', '##az', '##li', 'mahmud', '<', 'sy', '##az', '@', 'yahoo', '##com', '>', 'http', ':', '/', '/', 'news', '##asia', '##one', '##com', '/', 'news', '/', 'asia', '##one', '%', '2', '##bn', '##ew', '##s', '/', 'malaysia', '/', 'story', '/', 'a1', '##stor', '##y', '##200', 'internet', 'is', 'useful', 'for', 'making', 'the', 'truth', 'known', ',', 'says', 'dr', 'maha', '##thi', '##r', '`', '`', 'kuala', 'lumpur', ',', 'june', '1', '-', 'l', '##rb', '-', '[E11]', 'bern', '##ama', '[E12]', '-', 'rr', '##b', '-', '-', '[SEP]']


In [70]:
tokenizer.decode(tokens)

'[CLS] 2009 - 06 - 03t04 : 59 : 10 com > wrote : mohd syazli mahmud < syaz @ yahoocom > http : / / wwwyoutubecom / watch? v = qe8mi5bw9dc mohd syazli mahmud < syaz @ yahoocom > http : / / newsasiaonecom / news / asiaone % 2bnews / malaysia / story / a1story200 internet is useful for making the truth known, says dr mahathir ` ` kuala lumpur, june 1 - lrb - [E11] bernama [E12] - rrb - - [SEP]'

In [42]:
cnt = 0
for x in train_data:
    if 30524 not in x['tokens']:
       cnt += 1 

In [43]:
cnt

0

In [ ]:
train_data[1]

In [72]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(config.bert_path, additional_special_tokens=["[E11]", "[E12]", "[E21]", "[E22]"])
tokenizer.convert_tokens_to_ids("[MASK]")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


103

In [98]:
from transformers import BertForMaskedLM
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
text = ["[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [MASK] created the Muppets . [SEP]", "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [MASK] created the Muppets . [SEP]"]

input_ids = tokenizer.encode(text, return_tensors="pt")
mask_token_index = torch.where(input_ids == tokenizer.mask_token_id)
output = model(input_ids, return_dict=True)
logits = output.logits


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [102]:
input_ids

tensor([[101, 100, 100, 102]])

In [100]:
tokens = input_ids.tolist()
mask_idx = np.argwhere(np.array(tokens) == 103)[0][0]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [96]:
mask_output = []

for i in range(input_ids.shape[0]):
    instance_output = torch.index_select(logits, 0, torch.tensor(i))
    instance_output = torch.index_select(instance_output, 1, torch.tensor(mask_idx))
    mask_output.append(instance_output)
mask_output = torch.cat(mask_output, dim=0)
    
    

In [97]:
mask_output.shape

torch.Size([1, 1, 30522])

In [8]:
import model.bert_encoder
from model.bert_encoder import Bert_EncoderMLM
config.pattern = "entity_marker_mask"
model = Bert_EncoderMLM(config)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


AssertionError: Torch not compiled with CUDA enabled

In [5]:
from config import Config
config = Config('config.ini')

In [6]:
config.infonce_temprature

AttributeError: 'Config' object has no attribute 'infonce_temprature'

In [2]:
additional_special_tokens = ["[E11]", "[E12]", "[E21]", "[E22]"]
additional_special_tokens.extend([f"[REL{i}]" for i in range(1, 50 + 1)])

In [7]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', additional_special_tokens=additional_special_tokens)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
tokenizer.convert_tokens_to_ids("[REL1]")

30526

In [4]:
import torch
import torch.nn as nn
m = nn.Softmax(dim=1)
input = torch.randn(2, 3) + 1e8
output = m(input)

In [5]:
output

tensor([[0.3333, 0.3333, 0.3333],
        [0.3333, 0.3333, 0.3333]])

In [8]:
V = torch.rand(1,3000)
C = torch.rand(5,768)
W = torch.rand(3000,768)
output = torch.matmul(V,W)
output = torch.matmul(output,C.T)
output.shape


torch.Size([1, 5])

In [14]:
a = torch.rand(1,1)
b = torch.rand(1,5)
temp = torch.cat([a,b],dim=1).squeeze()

In [17]:
m = nn.Softmax(dim=0)
output = m(temp)

In [18]:
output

tensor([0.1380, 0.1012, 0.2047, 0.1749, 0.1983, 0.1830])

In [19]:
a = torch.tensor(1.0)
b = torch.rand(3)

In [24]:
torch.cat([a.unsqueeze(0),b],dim=0)

tensor([1.0000, 0.0674, 0.6470, 0.4542])

In [31]:
h = torch.tensor([61.01224136352539 ,-23.5942, 162.7764,  70.3629, -92.8090,  34.2552])
m(h/abs(h).max())

tensor([0.1736, 0.1032, 0.3244, 0.1839, 0.0675, 0.1473])

In [30]:
h/abs(h).max()

tensor([ 0.3748, -0.1449,  1.0000,  0.4323, -0.5702,  0.2104])

In [29]:
abs(h)

tensor([ 61.0122,  23.5942, 162.7764,  70.3629,  92.8090,  34.2552])

In [1]:
from transformers import BertTokenizer , BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
a = ["This is a sample text", "This is another sample text"]
inputs = tokenizer(a, return_tensors="pt")

In [12]:
outputs = model(inputs['input_ids'])

In [17]:
outputs[0].shape

torch.Size([2, 7, 768])